# Unitary Decomposition Based on QuCT

**Author:** Congliang Lang & Siwei Tan  

**Date:** 8/4/2024

Based on paper "[QuCT: A Framework for Analyzing Quantum Circuit by Extracting Contextual and Topological Features][1]" (MICRO 2023)

[1]: https://dl.acm.org/doi/10.1145/3613424.3614274

The expressiveness of gate vectors enables QuCT to reconstruct sub-circuits via gate vectors, which is a necessary ability for tasks involving circuit generation, such as the unitary decomposition. Unitary decomposition is a task that takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.

In [1]:
import os
os.chdir("..")
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

import time
import ray
ray.init(log_to_driver=False)
from qiskit.quantum_info import random_unitary

from janusq.data_objects.circuit import qiskit_to_circuit
from janusq.data_objects.random_circuit import random_circuits
from janusq.data_objects.backend import  LinearBackend
from janusq.analysis.vectorization import RandomwalkModel
from janusq.analysis.unitary_decompostion import decompose

2025-01-09 18:58:08,392	INFO worker.py:1724 -- Started a local Ray instance.


## Construct U2V model

The U2V model serves as the bridge between unitaries and gate vectors, where the sub-circuits reconstructed from these candidate vectors will replace the search space of QFAST. To build such a model, we obtain a U2V dataset composed of <$unitary,~\{vectors\}$> pairs, derived from a set of random circuits generated with the same scheme mentioned in Section~\ref{sec:equ_prediction}.

In [2]:
n_qubits = 5
backend = LinearBackend(n_qubits, 1, basis_two_gates = ['crz'])

In [3]:
from janusq.analysis.unitary_decompostion import U2VModel

n_step = 2

dataset = random_circuits(backend, n_circuits=50, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=False)

up_model = RandomwalkModel(
    n_step, 4 ** n_step, backend, directions=('parallel', 'next'))
up_model.train(dataset, multi_process=True,
                        remove_redundancy=False)

u2v_model = U2VModel(up_model)
data = u2v_model.construct_data(dataset, multi_process=False)
u2v_model.train(data, n_qubits)


100%|██████████| 6/6 [00:01<00:00,  4.19it/s]


len(Us) =  964 len(gate_vecs) =  964
Start construct U2VMdoel
Finish construct U2VMdoel, costing 19.746076107025146s


## Decompose a Unitary

In [4]:
# generate a random unitary
unitary = random_unitary(2**n_qubits).data

# apply decomposition
start_time = time.time()
quct_circuit = decompose(unitary, allowed_dist = 0.2, backend = backend, u2v_model = u2v_model, multi_process = True)
quct_time = time.time() - start_time

print(quct_time)
print(quct_circuit)

100%|██████████| 1/1 [2:10:48<00:00, 7848.87s/it]


10882.373190879822
     ┌────────────────────────────┐ ░                ░ »
q_0: ┤ U(2.3495,-0.80157,0.82685) ├─░───────■────────░─»
     └┬──────────────────────────┬┘ ░       │        ░ »
q_1: ─┤ U(3.2819,1.5129,-2.1326) ├──░───────┼────────░─»
     ┌┴──────────────────────────┤  ░       │        ░ »
q_2: ┤ U(0.30684,-1.2937,1.4175) ├──░───────┼────────░─»
     ├───────────────────────────┴┐ ░ ┌─────┴──────┐ ░ »
q_3: ┤ U(2.3369,-1.1516,-0.36491) ├─░─┤ Rz(3.2855) ├─░─»
     └────────────────────────────┘ ░ └────────────┘ ░ »
q_4: ───────────────────────────────░────────────────░─»
                                    ░                ░ »
«     ┌────────────────────────────┐ ░                 ░ »
«q_0: ┤ U(2.158,-0.39564,-0.80157) ├─░────────■────────░─»
«     └────────────────────────────┘ ░ ┌──────┴──────┐ ░ »
«q_1: ───────────────────────────────░─┤ Rz(-2.6789) ├─░─»
«                                    ░ └─────────────┘ ░ »
«q_2: ───────────────────────────────░─────────────────░─»


In [5]:
# compare it with the qsd method
from qiskit.synthesis.unitary.qsd import qs_decomposition

start_time =time.time()
qc = qs_decomposition(unitary)

qsd_circuit = qiskit_to_circuit(qc)
qsd_time = time.time() - start_time

print(qsd_time)
print(qc)

/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


0.4720780849456787
global phase: 4.5085
        ┌──────────────┐                                                      »
q0_0: ──┤0             ├───■───────────────────────────────────────────────■──»
        │  circuit-891 │   │                                               │  »
q0_1: ──┤1             ├───┼───────────────────────■───────────────────────┼──»
       ┌┴──────────────┤ ┌─┴─┐┌─────────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐»
q0_2: ─┤ U(0,0,1.4515) ├─┤ X ├┤ U(0,0,-0.84341) ├┤ X ├┤ U(0,0,-0.51602) ├┤ X ├»
       ├───────────────┴┐└───┘└─────────────────┘└───┘└─────────────────┘└───┘»
q0_3: ─┤ U(0,0,0.64008) ├─────────────────────────────────────────────────────»
      ┌┴────────────────┤                                                     »
q0_4: ┤ U(0,0,-0.42077) ├─────────────────────────────────────────────────────»
      └─────────────────┘                                                     »
«                                ┌──────────────┐                   »
«q0_0: ───

In [6]:
synthesis_method_result = [qsd_circuit,  quct_circuit]
synthesis_method_time = [qsd_time,  quct_time]
for res, tim in zip(synthesis_method_result, synthesis_method_time):
    print(f"#gate: {res.n_gates}, #two_qubit_gate: {res.num_two_qubit_gate}, depth: {res.depth}, time: {tim} \n")

#gate: 897, #two_qubit_gate: 379, depth: 815, time: 0.4720780849456787 

#gate: 647, #two_qubit_gate: 213, depth: 374, time: 10882.373190879822 

